In [11]:
from mesa.batchrunner import BatchRunner
from mesa.batchrunner import FixedBatchRunner
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from SALib.sample import saltelli
from SALib.analyze import sobol
from Simulate import Simulation


plt.style.use('default')

# Sobol SA

Sobol Sensitivity Analysis ([Sobol 2001](http://www.sciencedirect.com/science/article/pii/S0378475400002706), [Saltelli 2002](http://www.sciencedirect.com/science/article/pii/S0010465502002801), [Saltelli et al. 2010](http://www.sciencedirect.com/science/article/pii/S0010465509003087)) is a global SA method that determines the contribution of each input parameter or a combination of parameters and their interaction to the overall output variance. OFAT, while it is easier to implement (and certainly is less thought-intensive), has a couple of downsides.

1. OFAT requires a large amount of runs to get accurate results
2. OFAT cannot estimate interactions of combinations of inputs
3. OFAT can miss optimal settings of factors

Sobol can find higher order interactions, but still requires a large amount of runs.

### Getting the data

Before we can start analysing the model, we will have to sample our data. There are multiple methods for sampling included in SALib, but since we're using Sobol, we will use Saltelli sampling for this. 

The following code shows how you could collect data for the "wolf-sheep" model.

In [19]:
# We define our variables and bounds
problem = {
    'num_vars': 3,
    'names': ['ferry_base_price', 'capacity', 'ferry_base_time'],
    'bounds': [[1, 10], [1, 10], [1, 10]]
}

metrics = [
        'Ferry_Island_A_Island_B_users',
        'Ferry_Island_B_Island_A_users', 
        'Speedboat_Island_A_Island_B_users', 
        'Speedboat_Island_B_Island_A_users'
    ]

# Set the repetitions, the amount of steps, and the amount of distinct values per variable
replicates = 30         # ?
max_steps = 100         # = num_days?
distinct_samples = 32   # 1024

param_values = saltelli.sample(problem, distinct_samples, calc_second_order=False)


# Set the outputs
model_reporters = {"Percentage_ferry_users": lambda m: m.return_percentage_ferry_users(metrics)}

data = {}

batch = BatchRunner(Simulation, 
                    max_steps=max_steps,
                    variable_parameters={name:[] for name in problem['names']},
                    model_reporters=model_reporters)

count = 0
data = pd.DataFrame(index=range(replicates*len(param_values)), 
                                columns=['Percentage_Ferry_users'])
data['Percentage_Ferry_users'] = None



for i, var in enumerate(problem['names']):
    # Get the bounds for this variable and get <distinct_samples> samples within this space (uniform)
    samples = np.linspace(*problem['bounds'][i], num=distinct_samples, dtype=int)
    
    # Keep in mind that wolf_gain_from_food should be integers. You will have to change
    # your code to acommodate for this or sample in such a way that you only get integers.
    # if var == 'wolf_gain_from_food':
    #     samples = np.linspace(*problem['bounds'][i], num=distinct_samples, dtype=int)
    
    batch = FixedBatchRunner(Simulation,
                        max_steps=max_steps,
                        iterations=replicates,
                        parameters_list=[{var: value} for value in samples],
                        fixed_parameters= None,
                        model_reporters=model_reporters,
                        display_progress=True)
    
    batch.run_all()
    
    data[var] = batch.get_model_vars_dataframe()
    

C:\Users\kevin\AppData\Local\Temp\ipykernel_10328\2146681405.py:20: DeprecationWarning: `salib.sample.saltelli` will be removed in SALib 1.5. Please use `salib.sample.sobol`
  param_values = saltelli.sample(problem, distinct_samples, calc_second_order=False)
0it [00:00, ?it/s]


AttributeError: 'Simulation' object has no attribute 'running'

In [18]:
# Set the repetitions, the amount of steps, and the amount of distinct values per variable
replicates = 10
max_steps = 100
distinct_samples = 10

# We get all our samples here
param_values = saltelli.sample(problem, distinct_samples, calc_second_order=False)

# READ NOTE BELOW CODE
batch = BatchRunner(Simulation, 
                    max_steps=max_steps,
                    variable_parameters={name:[] for name in problem['names']},
                    model_reporters=model_reporters)

count = 0
data = pd.DataFrame(index=range(replicates*len(param_values)), 
                                columns=['Price', 'Density', 'Travel_time'])
data['Run'], data['Sheep'], data['Wolves'] = None, None, None

for i in range(replicates):
    for vals in param_values: 
        # Change parameters that should be integers
        vals = list(vals)
        vals[2] = int(vals[2])
        # Transform to dict with parameter names and their values
        variable_parameters = {}
        for name, val in zip(problem['names'], vals):
            variable_parameters[name] = val

        batch.run_iteration(variable_parameters, tuple(vals), count)
        iteration_data = batch.get_model_vars_dataframe().iloc[count]
        iteration_data['Run'] = count # Don't know what causes this, but iteration number is not correctly filled
        data.iloc[count, 0:3] = vals
        data.iloc[count, 3:6] = iteration_data
        count += 1

        clear_output()
        print(f'{count / (len(param_values) * (replicates)) * 100:.2f}% done')

C:\Users\kevin\AppData\Local\Temp\ipykernel_10328\1324753691.py:7: DeprecationWarning: `salib.sample.saltelli` will be removed in SALib 1.5. Please use `salib.sample.sobol`
  param_values = saltelli.sample(problem, distinct_samples, calc_second_order=False)
c:\Users\kevin\anaconda3\envs\ABM\Lib\site-packages\SALib\sample\saltelli.py:109: UserWarning: 
        Convergence properties of the Sobol' sequence is only valid if
        `N` (10) is equal to `2^n`.
        
  warnings.warn(msg)


AttributeError: 'Simulation' object has no attribute 'running'

Note that even though we use the BatchRunner provided by Mesa, we do not use its full capabilities. Normally, you would set all parameters properly (at line 25) and then run the batchrunner with `batch.run_all()`. However, the batchrunner will then proceed to run every possible combination of the variables you have passed it. We already have the combinations (samples) we need, because we got those from SALib.

Preferably, you would want to save the results as a csv so you can load them more easily when analysing your results. 

Printing the data shows (a part of) our results:

As you can see we have 800 distinct results. We have 3 variables, and we take 2 distinct values per sample per variable. A single sample from saltelli-sampling results in $2D+2$ different combinations for $D$ variables, $2(3)+2=8$ for our case (for more details, take a look at [Saltelli et al. 2010](http://www.sciencedirect.com/science/article/pii/S0010465509003087) and the [saltelli.sample() documentation](http://salib.readthedocs.io/en/latest/_modules/SALib/sample/saltelli.html)). Since we take $N=10$ samples, we get $N*(2D+2)=80$ combinations. We repeat each of these combinations 10 times, resulting in 800 results.
Note that the above analysis includes second-order indices. For a case where you only wish to study first- and total-order effects, `calc_second_order` for `saltelli.sample()` is set to false, and a single sample is sufficiently composed of $D+2$ different combinations.

We only consider 2 outputs in this example; the amount of sheep and wolves at the end of the simulation. Can you think of more outputs?

### Analyzing

Now we can use the `analyze()` method provided by SALib that performs Sobol analysis. 

In [ ]:
Si_Ferry_users = sobol.analyze(problem, data['Sheep'].values, print_to_console=True)

This is not very insightfull. Let's make a function that can plot this.

In [ ]:
def plot_index(s, params, i, title=''):
    """
    Creates a plot for Sobol sensitivity analysis that shows the contributions
    of each parameter to the global sensitivity.

    Args:
        s (dict): dictionary {'S#': dict, 'S#_conf': dict} of dicts that hold
            the values for a set of parameters
        params (list): the parameters taken from s
        i (str): string that indicates what order the sensitivity is.
        title (str): title for the plot
    """

    if i == '2':
        p = len(params)
        params = list(combinations(params, 2))
        indices = s['S' + i].reshape((p ** 2))
        indices = indices[~np.isnan(indices)]
        errors = s['S' + i + '_conf'].reshape((p ** 2))
        errors = errors[~np.isnan(errors)]
    else:
        indices = s['S' + i]
        errors = s['S' + i + '_conf']
        plt.figure()

    l = len(indices)

    plt.title(title)
    plt.ylim([-0.2, len(indices) - 1 + 0.2])
    plt.yticks(range(l), params)
    plt.errorbar(indices, range(l), xerr=errors, linestyle='None', marker='o')
    plt.axvline(0, c='k')

We'll first plot 1st, 2nd, and total-order sensitivity for the output variable 'Sheep', then for 'Wolves'. 

In [ ]:
for Si in (Si_sheep, Si_wolves):
    # First order
    plot_index(Si, problem['names'], '1', 'First order sensitivity')
    plt.show()

    # Second order
    plot_index(Si, problem['names'], '2', 'Second order sensitivity')
    plt.show()

    # Total order
    plot_index(Si, problem['names'], 'T', 'Total order sensitivity')
    plt.show()